In [4]:
!pip install mrjob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 kB 6.6 MB/s eta 0:00:00


In [13]:
%%file mr_word_count.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import re

def split_properley (line):

    title = None
    genres = None
    indices = [0]

    if "\"" in line: # Means we have multiple entries (commas) in at least one field

        # Find the indices of all apostrophes in this line:
        indices = [index for index in range(len(line)) if line.startswith('\"', index)]

        # Only 'title' contains commas
        if indices[0] == 0 and len(indices) == 2: # Hint: Does this fit our current schema?
            title = line[indices[0]+1 : indices[1]]

        elif len(indices)==2: # Only 'genre' contain commas
            genres = line[ indices[0]+1 : indices[1] ]

        else: # Both contain commas
            title = line[indices[0]+1 : indices[1]]
            genres = line[ indices[2]+1 : indices[3] ]

    # Take the rest of the line after the last apostrophe:
    line_after = line[indices[-1]:].split(',')

    # Extract the remaining fields:
    date = line_after[-3]
    air_time = int(line_after[-2])

    if genres == None: # We only have one genre
        genres = line_after[-4]

    if title == None:
        title = line.split(',')[0] # Hint: Does this fit our current schema?

    genres = genres.split(',')

    return title, genres, air_time, date

data = sc.textFile('tiny_50k_daily_prog_data.csv.csv')
header = data.first()

class MRWordFrequencyCount(MRJob):
    def mapper(self, _, line):
        title, genres, air_time, date = split_properley (line)
        if line != header:
          if int(70000 <= air_time < 90000):
            for l in ['j','q','z']:
              if l in title.lower():
                for k in ['Sitcom', 'Talk', 'Politics', 'Spanish', 'Community', 'Martial arts']:
                  if k in genres:
                    yield ((title, date, genres), 1)

    def combiner(self, word, counts):
        # optimization: sum the words we've seen so far
        yield (word, sum(counts))

    def reducer(self, word, counts):
        # send all (word, num_occurrences) pairs to the same reducer.
        # num_occurrences is so we can easily use Python's max() function.
        yield word, sum(counts)


if __name__ == '__main__':
    MRWordFrequencyCount.run()

Overwriting mr_word_count.py


In [14]:
!python mr_word_count.py tiny_50k_daily_prog_data.csv.csv -q

Traceback (most recent call last):
  File "/content/mr_word_count.py", line 45, in <module>
    data = sc.textFile('tiny_50k_daily_prog_data.csv.csv')
NameError: name 'sc' is not defined
